# Mongabay basemaps and layers

In [1]:
import ipyleaflet as ipyl
import ee

ee.Initialize()

## Basemaps

In [15]:
mongabay_paper_url = "https://api.mapbox.com/styles/v1/mongabay/ckai1bze50hmj1ipohkupl0qo/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoibW9uZ2FiYXkiLCJhIjoiY2s4N25oZG82MGFyejNsb2lnN3ZrbG1jbyJ9.coW3do99wi_PmnW6OylFbA"
#'https://api.mapbox.com/styles/v1/mongabay/ck8egahvy26m01hnv9f7i1cf3/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoibW9uZ2FiYXkiLCJhIjoiY2s4N25oZG82MGFyejNsb2lnN3ZrbG1jbyJ9.coW3do99wi_PmnW6OylFbA'
mongabay_paper_basemap = ipyl.TileLayer(
    url=topography_url,
    layers="topography",
    format="image/png",
    name="Topography",
    opacity=1,
)


planet_url = "https://tiles.planet.com/basemaps/v1/planet-tiles/global_monthly_2020_01_mosaic/gmap/{z}/{x}/{y}.png?api_key=f366791cb438466fb9e3e492e721da8d"
planet_basemap = ipyl.TileLayer(
    url=planet_url, layers="planet", format="image/png", name="Planet", opacity=1
)

landsat_url = (
    "https://storage.cloud.google.com/skydipper-landsat-cache/2018/{z}/{x}/{y}.png"
)
landsat_basemap = ipyl.TileLayer(
    url=landsat_url, layers="landsat", format="image/png", name="Landsat", opacity=0.5
)

canopy_height_url = "https://api.mapbox.com/v4/mongabay.0rgyb3bx/{z}/{x}/{y}.png?access_token=pk.eyJ1IjoibW9uZ2FiYXkiLCJhIjoiY2s4N25oZG82MGFyejNsb2lnN3ZrbG1jbyJ9.coW3do99wi_PmnW6OylFbA"
canopy_height_layer = ipyl.TileLayer(
    url=canopy_height_url,
    layers="canopy_height",
    format="image/png",
    name="Global Forest Canopy Height",
    opacity=1,
)

In [ ]:
[
    {
        "id": "gain",
        "name": "Forest canopy height",
        "config": {
            "type": "raster",
            "source": {
                "type": "raster",
                "tiles": [
                    "https://api.mapbox.com/v4/mongabay.0rgyb3bx/{z}/{x}/{y}.png?access_token=pk.eyJ1IjoibW9uZ2FiYXkiLCJhIjoiY2s4N25oZG82MGFyejNsb2lnN3ZrbG1jbyJ9.coW3do99wi_PmnW6OylFbA"
                ],
                "minzoom": 3,
                "maxzoom": 12,
            },
        },
        "legendConfig": {
            "items": [
                {"color": "#ffffff", "value": "0"},
                {"color": "#E7F1DB", "value": "5"},
                {"color": "#C2DCA3", "value": "10"},
                {"color": "#9EC76E", "value": "20"},
                {"color": "#79B137", "value": "35"},
                {"color": "#559C01", "value": "50 m"},
            ],
            "type": "gradient",
            "unit": "m",
        },
    }
]

In [16]:
center = (0, 0)
m = ipyl.Map(center=(center), zoom=2)


m.add_layer(mongabay_paper_basemap)
m.add_layer(planet_basemap)
m.add_layer(landsat_basemap)
m.add_layer(canopy_height_layer)

control = ipyl.LayersControl(position="topright")
m.add_control(control)
m.add_control(ipyl.FullScreenControl())
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

**Global Forest Height**

In [4]:
name = "Global Forest Height"

image = ee.Image("users/iker/Mongabay/3DGlobalVeg")

hansenImage = ee.Image("UMD/hansen/global_forest_change_2015")

# Select the land/water mask.
datamask = hansenImage.select("datamask")

# Create a binary mask.
mask = datamask.eq(1)

# Update the image with the water mask.
image = image.updateMask(mask)

# Make a water image out of the mask.
water = mask.Not()

# Mask water with itself to mask all the zeros (non-water)..
water = water.mask(water)

sld_interval = (
    "<RasterSymbolizer>"
    + '<ColorMap extended="false" type="ramp">'
    + '<ColorMapEntry color="#ffffff" quantity="0"  opacity="0" />'
    + '<ColorMapEntry color="#E7F1DB" quantity="5"  opacity="1" />'
    + '<ColorMapEntry color="#C2DCA3" quantity="10"  />'
    + '<ColorMapEntry color="#9EC76E" quantity="20" />'
    + '<ColorMapEntry color="#79B137" quantity="35"  />'
    + '<ColorMapEntry color="#559C01" quantity="50"  />'
    + "</ColorMap>"
    + "</RasterSymbolizer>"
)

# Make a mosaic of images.
mosaic = ee.ImageCollection(
    [
        image.sldStyle(sld_interval)
        # water.visualize(palette = '000044'),
    ]
).mosaic()

In [5]:
ee_tiles = (
    "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
)

m = ipyl.Map(center=(0, 0), zoom=2)

mapid = mosaic.getMapId()
tiles_url = ee_tiles.format(**mapid)

tile_layer = ipyl.TileLayer(url=tiles_url, format="image/png", name=name, opacity=1)

m.add_layer(topography_basemap)
m.add_layer(tile_layer)

control = ipyl.LayersControl(position="topright")
m.add_control(control)
m.add_control(ipyl.FullScreenControl())

m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [7]:
def get_tiles(
    images, sld_intervals, names, basemap="topography", center=(0, 0), zoom=2
):
    ee_tiles = (
        "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    )

    m = ipyl.Map(center=(center), zoom=zoom)

    if basemap == "topography":
        m.add_layer(topography_basemap)
    if basemap == "landsat":
        m.add_layer(landsat_basemap)

    for n, image in enumerate(images):
        image = image.sldStyle(sld_intervals[n])
        mapid = image.getMapId()
        tiles_url = ee_tiles.format(**mapid)

        tile_layer = ipyl.TileLayer(
            url=tiles_url, format="image/png", name=names[n], opacity=1
        )

        m.add_layer(tile_layer)

    control = ipyl.LayersControl(position="topright")
    m.add_control(control)
    m.add_control(ipyl.FullScreenControl())

    return m